In [3]:
# Import and load the spacy model
import spacy
nlp=spacy.load("en_core_web_sm") 

# Getting the ner component
ner=nlp.get_pipe('ner')

In [11]:
import pandas as pd
df = pd.read_json('all.jsonl', lines=True)

In [14]:
df = df[['data', 'label']]

## CASE1

In [51]:
import sys
import json

TRAIN_DATA = []
fo = open('all.jsonl', "r")
lines = fo.readlines()
for line in lines:
    line =json.loads(line)
    if "label" in line:
        line["entities"] = line.pop("label")
    else:
        line["entities"] = []

    tmp_ents = []
    for e in line["entities"]:
        if e[2] in ['make', 'model', 'trim']:
             tmp_ents.append({"start": e[0], "end": e[1], "label": e[2]})
        line["entities"] = tmp_ents
    if (len(line["data"]) > 5):
        TRAIN_DATA.append(json.dumps({"entities": line["entities"], "text": line["data"]}))

## CASE2

In [55]:
import json
labeled_data = []
with open(r"all.jsonl", "r") as read_file:
    for line in read_file:
        data = json.loads(line)
        labeled_data.append(data)

In [56]:
TRAINING_DATA = []
for entry in labeled_data:
    entities = []
    for e in entry['label']:
        entities.append((e[0], e[1],e[2]))
    spacy_entry = (entry['data'], {"entities": entities})
    TRAINING_DATA.append(spacy_entry)

In [61]:
import spacy
import random
import json
nlp = spacy.blank("en")
# ner = nlp.create_pipe("ner")
ner= nlp.add_pipe('ner')
ner.add_label("make")
ner.add_label("model")
ner.add_label("trim")
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [66]:
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :
    sizes = compounding(1.0, 4.0, 1.001)
    # Training for 30 iterations     
    for itn in range(30):
        # shuffle examples before training
        random.shuffle(TRAIN_DATA)
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=sizes)
        # ictionary to store losses
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            # Calling update() over the iteration
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

ValueError: too many values to unpack (expected 2)

In [79]:
import spacy
import random
import json
from spacy.training.example import Example
nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe('ner')
ner.add_label("make")
ner.add_label("model")
ner.add_label("trim")
# Start the training
nlp.begin_training()
# Loop for 40 iterations
for itn in range(40):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}
# Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            # Update the model
            nlp.update([example], losses=losses, drop=0.3)
    print(losses)

/usr/local/lib/python3.9/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "For Sale 2006 Toyota Tundra SR5 V8 Cyl 4.7L Engine..." with entities "[(14, 20, 'make'), (20, 27, 'model'), (28, 50, 'tr...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "2006 Chrysler 300C..HEMI..5.7L HEMI , engine ..Cre..." with entities "[(5, 13, 'make'), (14, 17, 'model'), (17, 18, 'tri...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be ali

{'ner': 784.5292292747996}
{'ner': 81.73998670393244}
{'ner': 163.63509768066046}
{'ner': 53.85248123450398}
{'ner': 41.301099149019}
{'ner': 31.578431538221285}
{'ner': 31.02927501548565}
{'ner': 23.246153879630988}
{'ner': 311.51248985723635}
{'ner': 21.986876120889264}
{'ner': 21.456929596801967}
{'ner': 14.024556979763664}
{'ner': 261.0705402618238}
{'ner': 11.162653316814271}
{'ner': 8.912764354994595}
{'ner': 82.6486727700741}
{'ner': 92.28457846971841}
{'ner': 15.144224153817033}
{'ner': 172.91638540608275}
{'ner': 6.114402283857028}
{'ner': 71.66545406056747}
{'ner': 10.722857371182746}
{'ner': 28.68005498047332}
{'ner': 6.9262309984436365}
{'ner': 2.248697395148905}
{'ner': 1.907833984598222}
{'ner': 5.865848606273045}
{'ner': 0.08808825793503126}
{'ner': 0.0026773613635440578}
{'ner': 1.9814335421285774}
{'ner': 6.709283773554105e-05}
{'ner': 30.624863298625964}
{'ner': 2.3893173630603304}
{'ner': 0.004014170427255903}
{'ner': 0.07391437584235437}
{'ner': 0.02629792134753726}

In [93]:
from spacy import displacy
example = "2008 Subaru Impreza coupe AWD 5SP manual 176k runs great, clean title, clean car check $4995 call 949 five four seven 6400 DLR"

In [94]:
doc = nlp(example)
displacy.render(doc, style='ent')

In [90]:
for ent in doc.ents:
    print(ent.label_ + ":", ent.text)

In [96]:
nlp.to_disk('models/mca-ner-scratch.model')